In [1]:
from nemo_text_processing.text_normalization.sv.taggers.cardinal import CardinalFst as tnCard
from nemo_text_processing.inverse_text_normalization.sv.taggers.cardinal import CardinalFst as Card
from nemo_text_processing.text_normalization.sv.taggers.decimal import DecimalFst as tnDec
from nemo_text_processing.inverse_text_normalization.sv.taggers.decimal import DecimalFst as Dec

In [2]:
tn_card = tnCard(False)
card = Card(tn_card)
tn_dec = tnDec(tn_card, False)
dec = Dec(card, tn_dec)

In [3]:
("en miljon" @ dec.fst).string()

'decimal { integer_part: "1" quantity: "miljon" preserve_order: true }'

In [4]:
("ett miljon" @ dec.fst).string()

'decimal { integer_part: "1" quantity: "miljon" preserve_order: true }'

In [5]:
from nemo_text_processing.inverse_text_normalization.sv.verbalizers.decimal import DecimalFst as vDec

In [6]:
v_dec = vDec()

In [7]:
('decimal { integer_part: "2" quantity: "miljon" preserve_order: true }' @ v_dec.fst).string()

'2 miljon'

In [8]:
('decimal { integer_part: "2" preserve_order: true }' @ v_dec.fst).string()

'2'

In [9]:
("ett komma tre" @ dec.fst).string()

'decimal { integer_part: "1" fractional_part: "3" preserve_order: true }'

In [10]:
('decimal { integer_part: "1" fractional_part: "3" preserve_order: true }' @ v_dec.fst).string()

'1,3'

In [11]:
('decimal { integer_part: "1" fractional_part: "5" quantity: "miljoner" preserve_order: true }' @ v_dec.fst).string()

'1,5 miljoner'

In [12]:
("ett komma tre miljoner" @ dec.fst).string()

ERROR: StringFstToOutputLabels: State 60 has multiple outgoing arcs


FstOpError: Operation failed

In [13]:
("ett komma tre miljoner" @ dec.fst)